In [202]:
%run gower_function-v6.4.ipynb
from pymongo import MongoClient
from random import randint
import pandas as pd
import numpy as np
import re
import category_encoders as ce
from scipy.spatial.distance import pdist,squareform, jaccard, cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
import category_encoders as ce

In [203]:
def get_data():
    client = MongoClient(port=27017)
    db = client.weare    
    student_queries = db.students.find({})
    user_queries = db.users.find({})
    students = pd.DataFrame(list(student_queries))
    users = pd.DataFrame(list(user_queries))
    return students, users
students, users = get_data()

In [204]:
def clean_channels(channel_obj_array):
    channels = []
    if isinstance(channel_obj_array, str):
        channel_obj_array = json.loads(channel_obj_array) 
    for channel_obj in channel_obj_array:
        if isinstance(channel_obj, collections.Mapping):
            channels.append(channel_obj['cid'])
            continue
        if isinstance(channel_obj, str):
            channels.append(channel_obj)
            continue
        raise Exception('Invalid type')
    return channels
users.channels = users.channels.apply(clean_channels)

In [205]:
def add_binarized_channels(users):
    binarized_channels = users.channels.apply('|'.join).str.get_dummies()
    col_list = list(binarized_channels.columns)
    column_mappings = {name: f'channel_{i}' for i, name in enumerate(col_list)}
    binarized_channels = binarized_channels.rename(index=str, columns=column_mappings)
    binarized_channels = binarized_channels.reset_index() 
    users = pd.merge(users, binarized_channels, left_index=True, right_index=True)
    users = users.drop('channels', 1)
    return users
users = add_binarized_channels(users)

In [206]:
def drop_student_columns(students):
    students['id']=students.Email.str.split("@", n=1, expand=True)[0]
    if '_id' in students.columns:
        del students['_id']
    #drop rows with 5 or more null values
    students.dropna(thresh=5, inplace=True)
    students = students.replace('', np.nan)
    #number of courses, credits are too dirty as many entered text instead of a number
    drop_columns = ["surveyCompletion", "Duration (in seconds)", "NumCourses", "transferCredits", "NumTranCredits",
                    "WhyProfile", "otherChannels", "otherEmploy", "NonUS", "otherIndusry", "KnownThroughProfile",
                    "otherEth", "PPLinPerson", "otherEmail"]
    #only drop these columns if they actually exist in the data
    drop_columns = [col for col in drop_columns if col in students.columns]
    students.drop(columns=drop_columns, inplace=True)
    return students
students = drop_student_columns(students)

In [207]:
def enumerate_ordinal_columns(students):
    #names of all the soc columns
    SOCs = ['SOC'+str(x+1) for x in range(10)]
    #names of all the CCE columns
    CCEs = ['CCE'+str(x+1) for x in range(24)]
    #mapping from each text column to ordinal values
    #format is array of dicts containing keys 'col' and 'mapping'
    ordinal_cols_mapping = []
    #the following are mappings from textual column values to numberical ones
    important_scale = [
            ('Extremely important', 7),
            ('Pretty important', 6),
            ('Fairly important', 5),
            ('Moderately important', 4),
            ('Somewhat important', 3),
            ('A little important', 2),
            ('Not at all important', 1)
    ]

    interest_scale = [
        ('Extremely interested', 5),
        ('Rather interested', 4),
        ('Somewhat interested', 3),
        ('A bit of interest', 2),
        ('Not at all interested', 1)
    ]

    Agree_scale = [
            ('Strongly agree', 5),
            ('Somewhat agree', 4),
            ('Neither agree nor disagree', 3),
            ('Somewhat disagree', 2),
            ('Strongly disagree', 1)
    ]
    #fill in the mapping array 
    for SOC in SOCs:
        ordinal_cols_mapping.append({
            "col": SOC,
            "mapping": Agree_scale
        })

    for CCE in CCEs:
        ordinal_cols_mapping.append({
            "col": CCE,
            "mapping": Agree_scale
        })

    for PeerInfo in ["PeerAvail", "PeerProfession", "PeerEdu", "PeerDetails"]:
        ordinal_cols_mapping.append({
            "col": PeerInfo,
            "mapping": important_scale
        })

    ordinal_cols_mapping.append({
        "col": "Mconnected",
        "mapping": interest_scale
    })
    encoder = ce.OrdinalEncoder(mapping=ordinal_cols_mapping, return_df=True)
    students = encoder.fit_transform(students)
    #Create new columns by taking the mean of related columns for each row
    students['SOC_f'] = students.apply(lambda row: np.mean([row['SOC9'], row['SOC10']]), axis=1)
    students['SOC_id'] = students.apply(lambda row: np.mean([row['SOC1'], row['SOC2'], row['SOC3'], row['SOC4'], row['SOC5']]), axis=1)
    students['CCE_IR'] = students.apply(lambda row: np.mean([row['CCE13'], row['CCE10'], row['CCE19'], row['CCE9']]), axis=1)
    students['CCE_Coor'] = students.apply(lambda row: np.mean([row['CCE24'], row['CCE20'], row['CCE22'], row['CCE21'], row['CCE17']]), axis=1)
    students['CCE_SS'] = students.apply(lambda row: np.mean([row['CCE1'], row['CCE2'], row['CCE3'], row['CCE4']]), axis=1)
    students.drop(columns=CCEs + SOCs, inplace=True)
    return students
students = enumerate_ordinal_columns(students)
students

,ActiveDuty,BWS,Degree,Email,First Name,IP Address,InUS,Last Name,Latitude,Longitude,...,industry,kids,marrital,progress,id,SOC_f,SOC_id,CCE_IR,CCE_Coor,CCE_SS
0,NaN,No,Master degree,sxa459@psu.edu,Sana,72.94.219.150,Yes,Abdulateef,40.351395,-75.930603,...,NaN,0,Married,14.0,sxa459,2.5,5.0,4.50,3.8,3.50
1,NaN,Yes,Undergraduate certificate,CFD5112@PSU.EDU,Christopher,50.251.57.49,Yes,Duntzee,40.103195,-75.338898,...,Other,0,Married,100.0,CFD5112,4.5,4.4,5.00,5.0,5.00
2,NaN,Yes,Master degree,MAG5188@PSU.EDU,Mindy,128.118.199.130,Yes,Barnett,40.784897,-77.847000,...,Educational services,0,Divorced,96.0,MAG5188,3.0,3.0,4.25,3.2,3.00
3,NaN,Yes,Bachelor degree,ama5044@psu.edu,Adrian,72.9.8.222,Yes,Alfano,38.290695,-76.522102,...,Management of companies or enterprises,0,Married,6.0,ama5044,1.5,4.0,4.50,3.8,4.50
4,NaN,Yes,Master degree,axg130@psu.edu,Adrienne,130.203.175.194,Yes,Darrah,40.807693,-77.861397,...,Educational services,1,Married,50.0,axg130,1.0,3.0,3.50,3.0,3.25
5,NaN,Yes,Bachelor degree,kub288@psu.edu,Kevin,107.11.81.182,Yes,Blodgett,39.955307,-82.071800,...,Mining or energy production,2,Married,24.0,kub288,1.5,3.6,3.00,2.6,3.75
6,NaN,Yes,Bachelor degree,AMB5012@PSU.EDU,Andrew,108.18.217.159,Yes,Brooks,38.817505,-77.630798,...,Other,3,Married,90.0,AMB5012,1.5,3.2,4.75,4.2,3.25
7,NaN,Yes,Bachelor degree,LQB5283@PSU.EDU,Luke,66.87.84.176,Yes,Bussnick,40.739105,-74.194504,...,Manufacturing,0,Single/Never Married,67.0,LQB5283,1.5,4.2,5.00,4.0,3.00
8,NaN,No,Master degree,mjg5978@psu.edu,Melinda,204.76.30.253,Yes,Baron,32.481400,-84.903297,...,Finance or insurance,0,Married,50.0,mjg5978,3.0,5.0,5.00,5.0,5.00
9,NaN,Yes,Bachelor degree,EKC5166@PSU.EDU,Emily,73.79.232.206,Yes,Chen,40.292099,-76.425903,...,Other,0,Single/Never Married,57.0,EKC5166,2.0,2.8,4.25,3.4,3.25


In [208]:
students

,ActiveDuty,BWS,Degree,Email,First Name,IP Address,InUS,Last Name,Latitude,Longitude,...,industry,kids,marrital,progress,id,SOC_f,SOC_id,CCE_IR,CCE_Coor,CCE_SS
0,NaN,No,Master degree,sxa459@psu.edu,Sana,72.94.219.150,Yes,Abdulateef,40.351395,-75.930603,...,NaN,0,Married,14.0,sxa459,2.5,5.0,4.50,3.8,3.50
1,NaN,Yes,Undergraduate certificate,CFD5112@PSU.EDU,Christopher,50.251.57.49,Yes,Duntzee,40.103195,-75.338898,...,Other,0,Married,100.0,CFD5112,4.5,4.4,5.00,5.0,5.00
2,NaN,Yes,Master degree,MAG5188@PSU.EDU,Mindy,128.118.199.130,Yes,Barnett,40.784897,-77.847000,...,Educational services,0,Divorced,96.0,MAG5188,3.0,3.0,4.25,3.2,3.00
3,NaN,Yes,Bachelor degree,ama5044@psu.edu,Adrian,72.9.8.222,Yes,Alfano,38.290695,-76.522102,...,Management of companies or enterprises,0,Married,6.0,ama5044,1.5,4.0,4.50,3.8,4.50
4,NaN,Yes,Master degree,axg130@psu.edu,Adrienne,130.203.175.194,Yes,Darrah,40.807693,-77.861397,...,Educational services,1,Married,50.0,axg130,1.0,3.0,3.50,3.0,3.25
5,NaN,Yes,Bachelor degree,kub288@psu.edu,Kevin,107.11.81.182,Yes,Blodgett,39.955307,-82.071800,...,Mining or energy production,2,Married,24.0,kub288,1.5,3.6,3.00,2.6,3.75
6,NaN,Yes,Bachelor degree,AMB5012@PSU.EDU,Andrew,108.18.217.159,Yes,Brooks,38.817505,-77.630798,...,Other,3,Married,90.0,AMB5012,1.5,3.2,4.75,4.2,3.25
7,NaN,Yes,Bachelor degree,LQB5283@PSU.EDU,Luke,66.87.84.176,Yes,Bussnick,40.739105,-74.194504,...,Manufacturing,0,Single/Never Married,67.0,LQB5283,1.5,4.2,5.00,4.0,3.00
8,NaN,No,Master degree,mjg5978@psu.edu,Melinda,204.76.30.253,Yes,Baron,32.481400,-84.903297,...,Finance or insurance,0,Married,50.0,mjg5978,3.0,5.0,5.00,5.0,5.00
9,NaN,Yes,Bachelor degree,EKC5166@PSU.EDU,Emily,73.79.232.206,Yes,Chen,40.292099,-76.425903,...,Other,0,Single/Never Married,57.0,EKC5166,2.0,2.8,4.25,3.4,3.25


In [209]:
def clean_kids_column(students):
    none_i = re.compile(r'none', flags=re.IGNORECASE)
    students['kids'].replace(none_i, 0, inplace=True)
    none_i = re.compile(r'zero', flags=re.IGNORECASE)
    students['kids'].replace(none_i, 0, inplace=True)
    stringany = re.compile(r'[a-zA-Z ()]+', flags=re.IGNORECASE)
    students.kids = students['kids'].replace(stringany, '')
    return students
students = clean_kids_column(students)

In [210]:
def fix_nan_columns(students):
    categorical_cols = ["gender", "InUS", "ethnicity", "Usstate", "marrital", "employment", "industry"]
    categorical_cols = [c for c in categorical_cols if c in students.columns]
    students_c_mode = students[categorical_cols].mode()
    print(f'mode listed are \n{students_c_mode.iloc[0]}')
    print(len(students_c_mode))
    for col in categorical_cols+['kids']:
        students[col].fillna(students[col].mode().iloc[0], inplace=True)
    # fill with mode, mean, or median
    students_mode, students_mean, students_median = students.mode().iloc[0], students.mean(), students.median()
    students.fillna(students_median, inplace=True)
    return students
students = fix_nan_columns(students)

mode listed are 
gender                         Female
InUS                              Yes
ethnicity                   Caucasian
Usstate                  Pennsylvania
marrital                      Married
employment    Working (paid employee)
industry         Educational services
Name: 0, dtype: object
1


In [211]:
def compute_distances(df, weights=None):
    numeric_columns = df._get_numeric_data().columns
    col_is_categorical = [col not in numeric_columns for col in df.columns]
    distances = gower_distances(df, categorical_features=col_is_categorical, feature_weight=weights)
    return distances
def get_matches(index, distances, df, num_matches=1):
    best_scores = sorted(distances[index])[1: 1+num_matches]
    best_entries = [np.where(distances[index]==score)[0][0] for score in best_scores]
    og_user = df.iloc[[index]]
    best_users = [df.iloc[[entry]] for entry in best_entries]
    all_users = [og_user, *best_users]
    concat = pd.concat(all_users)
    return concat
def display_matches(distances, df, num_matches=10):
    frame = get_matches(0, distances, df)
    for i in range(1, num_matches):
        match = get_matches(i, distances, df)
        frame = pd.concat([frame, match])
    return frame
def test_dataframe(df, weights=None):
    distances = compute_distances(df, weights=weights)
    return display_matches(distances, df)

In [214]:
test_dataframe(users)

/home/matt/.conda/envs/weare/lib/python3.7/site-packages/ipykernel_launcher.py:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


,_id,email,first_name,group,image_48,is_bot,is_custom_image,last_name,last_updated,local_area,...,channel_14,channel_15,channel_16,channel_17,channel_18,channel_19,channel_20,channel_21,channel_22,channel_23
0,ObjectId(5c399149ff0b61b77d00ce4d),bvh10@ist.psu.edu,Ben,3,https://secure.gravatar.com/avatar/28802966d0f...,False,None,Hanrahan,1504319350,America,...,0,0,0,0,0,0,0,0,0,0
2,ObjectId(5c399149ff0b61b77d00ce4f),tuy11@psu.edu,Tina,3,https://secure.gravatar.com/avatar/e74e1765013...,False,None,None,1504319351,America,...,0,0,0,0,0,0,0,0,0,0
1,ObjectId(5c399149ff0b61b77d00ce4e),yuw132@psu.edu,wuyu,2,https://secure.gravatar.com/avatar/6349a73089d...,False,None,None,1504319350,America,...,0,0,0,0,0,0,0,0,0,0
9,ObjectId(5c399149ff0b61b77d00ce5e),bikalpaneupane@ist.psu.edu,Bikalpa,2,https://secure.gravatar.com/avatar/11192bc82cc...,False,None,None,1504319351,America,...,0,0,1,0,0,0,0,0,0,0
2,ObjectId(5c399149ff0b61b77d00ce4f),tuy11@psu.edu,Tina,3,https://secure.gravatar.com/avatar/e74e1765013...,False,None,None,1504319351,America,...,0,0,0,0,0,0,0,0,0,0
0,ObjectId(5c399149ff0b61b77d00ce4d),bvh10@ist.psu.edu,Ben,3,https://secure.gravatar.com/avatar/28802966d0f...,False,None,Hanrahan,1504319350,America,...,0,0,0,0,0,0,0,0,0,0
3,ObjectId(5c399149ff0b61b77d00ce50),jomarabinda@gmail.com,jomarabinda,4,https://secure.gravatar.com/avatar/7392ec92f00...,False,None,None,1504319350,America,...,0,0,0,0,0,0,0,0,0,0
5,ObjectId(5c399149ff0b61b77d00ce56),adithya@pressmail.ch,adithyavr,1,https://secure.gravatar.com/avatar/8ef0409aac3...,False,None,None,1504319351,America,...,0,0,0,0,0,0,0,0,0,0
4,ObjectId(5c399149ff0b61b77d00ce55),dxc360@ist.psu.edu,Dong,1,https://secure.gravatar.com/avatar/2434caac082...,False,None,Chen,1504319351,America,...,0,0,0,0,0,0,0,0,0,0
5,ObjectId(5c399149ff0b61b77d00ce56),adithya@pressmail.ch,adithyavr,1,https://secure.gravatar.com/avatar/8ef0409aac3...,False,None,None,1504319351,America,...,0,0,0,0,0,0,0,0,0,0


In [212]:
%%latex
$$gower(\vec{a}, \vec{b})=\dfrac{\sum_{i=1}^{n}w_i * d(a_i, b_i)}{\sum_{i=1}^{n}w_i}$$
$$d(a,b) = \begin{cases}
\dfrac{|a-b|}{max(x_i) - min_i(x_i)}  & \text{if variable at $i$ is ordinal/continous} \\
\begin{cases}
0, & \text{if } a\neq b \\
1, & \text{if } a = b
\end{cases} & \text{if variable at $i$ is categorical}
\end{cases}$$

<IPython.core.display.Latex object>